# MultinomialNB における sample_weight

scikit-learn の多項ナイーブベイズモジュールに sample_weight を与えて動作確認します。

参考文献：
http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB

### テスト準備

長いので適宜読み飛ばしていただければ幸いです

In [1]:
from numpy import array
from scipy.sparse.csr import csr_matrix
tf_vector_array = array([
    [1, 0, 0, 5, 2], # １番目のクラス
    [0, 4, 0, 2, 2],
    [1, 2, 0, 4, 0], # <---２番目のクラスにも同じ特徴をもつものが混在
    [1, 2, 0, 6, 0],

    [1, 0, 0, 5, 0], # ２番目のクラス
    [0, 6, 0, 2, 2],
    [1, 2, 0, 4, 0], # <---１番目のクラスにも同じ特徴をもつものが混在
    [1, 2, 0, 5, 2],

    [0, 0, 1, 1, 0], # ３番目のクラス
    [0, 0, 1, 1, 0],
    [0, 0, 1, 1, 0]
    ])
tf_vector = csr_matrix(tf_vector_array) # これは特徴語の出現回数リストです。
tf_vector

<11x5 sparse matrix of type '<class 'numpy.int64'>'
	with 30 stored elements in Compressed Sparse Row format>

In [2]:
class_vector_array = array([
    1, # １番目のクラス
    1,
    1,
    1,

    2, # ２番目のクラス
    2,
    2,
    2,

    3, # ３番目のクラス
    3,
    3
    ])
class_vector_array # これは質問文ごとのクラスリスト（分類ラベルリスト）です。

array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3])

### 質問文を作成

訓練データに近い内容にしておきます

In [3]:
tf_vector_array_test = array([
    [1, 0, 0, 3, 2],  # これはクラス＝１想定
    [1, 2, 0, 4, 0],  # これはクラス＝２想定（ただし、同様の特徴を含むクラス１に誤選択される可能性あり）
    [0, 0, 1, 1, 0]   # これはクラス＝３想定
    ])
tf_vector_test = csr_matrix(tf_vector_array_test) # これは特徴語の出現回数リストです。
tf_vector_test.toarray()

array([[1, 0, 0, 3, 2],
       [1, 2, 0, 4, 0],
       [0, 0, 1, 1, 0]], dtype=int64)

### (2-a) sample_weight の指定を省略した場合

#### インスタンスを生成→学習実行

まずは、クラス間に類似特徴がまたがって混在した時の動きをみるため、重み付け定義を引数として指定しないようにします。

MultinomialNB インスタンスを生成し、学習を実行します。

In [4]:
from sklearn.naive_bayes import MultinomialNB

multinomialNB_noweight = MultinomialNB()
multinomialNB_noweight.fit(tf_vector, class_vector_array, sample_weight=None)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### 予測結果

２番目の質問に対し、２番目のクラスではなく<b>１番目のクラスが誤選択されてしまう</b>ことが確認できます。

In [5]:
Zp = multinomialNB_noweight.predict(tf_vector_test)
Zp

array([1, 1, 3])

### (2-b) sample_weight を指定した場合

訓練データごとに重み付け定義を指定します。

In [6]:
sample_weight = array([ # <---ご注意：sample_weight は array です
    1.0,
    1.0,
    0.3, # [1, 2, 0, 4, 0] という特徴を持った訓練データ（クラス＝１）の重み付けを下げます
    1.0,

    1.0,
    1.0,
    1.0,
    1.0,

    1.0,
    1.0,
    1.0
    ])
sample_weight # これは訓練データごとの重み付け定義リストです。

array([ 1. ,  1. ,  0.3,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ])



#### インスタンスを生成→学習

sample_weight を指定し、学習を実行します。

In [7]:
from sklearn.naive_bayes import MultinomialNB

multinomialNB = MultinomialNB()
multinomialNB.fit(tf_vector, class_vector_array, sample_weight=sample_weight)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### 予測の実行

２番目の質問文に対し、先ほどクラス＝１と回答されたところが、sample_weight 指定により<b>回答が回避されてしまう</b>ことが確認できます。

他の質問文は、概ね想定通りに分類されます。

In [8]:
Zp = multinomialNB.predict(tf_vector_test)
Zp

array([1, 2, 3])